In [1]:
# !sudo apt-get install -y fonts-nanum

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.ticker as ticker
import logging

logging.getLogger("matplotlib.font_manager").setLevel(logging.ERROR)

# fontpath = "/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf"
# fontprop = fm.FontProperties(fname=fontpath, size=12)
# plt.rcParams["font.family"] = fontprop.get_name()

# print(f"설정된 폰트: {fontprop.get_name()}")

In [3]:
!pip install sentencepiece # 미리 설치 안하면 아래에서 오류가 남

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.3 MB/s eta 0:00:00


In [4]:
import os
import re
import urllib.request
import zipfile
import sentencepiece as spm
import pandas as pd

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.__version__)

2.7.1+cu118


In [5]:
# def preprocess_korean(sentence):
#     sentence = sentence.strip()
#     sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
#     sentence = re.sub(r'\s+', ' ', sentence)
#     sentence = re.sub(r"[^가-힣a-zA-Z0-9?.!,\s]+", " ", sentence)
#     sentence = sentence.strip()
#     return sentence

# print("슝~")

In [6]:
def preprocess_korean(sentence):
    sentence = sentence.strip()
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    # sentence = re.sub(r'\s+', ' ', sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^가-힣a-zA-Z?.!,\s]+", " ", sentence)
    sentence = sentence.strip()
    return sentence

print("슝~")

슝~


In [7]:
import unicodedata

In [8]:
def preprocess_english(sentence):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)

    sentence = sentence.strip()

    return sentence

print("슝~")

슝~


In [9]:
def load_and_preprocess_data(ko_file_path, en_file_path, max_samples=None):
    """
    한국어-영어 병렬 데이터를 로드하고 전처리하는 함수
    
    Args:
        ko_file_path: 한국어 파일 경로
        en_file_path: 영어 파일 경로
        max_samples: 최대 샘플 수 (None이면 전체)
    
    Returns:
        pandas.DataFrame: 전처리된 데이터프레임
    """
    
    # 파일 읽기
    with open(ko_file_path, 'r', encoding='utf-8') as f:
        ko_lines = f.readlines()
    
    with open(en_file_path, 'r', encoding='utf-8') as f:
        en_lines = f.readlines()
    
    # 줄 수 맞추기 (더 짧은 쪽에 맞춤)
    min_len = min(len(ko_lines), len(en_lines))
    ko_lines = ko_lines[:min_len]
    en_lines = en_lines[:min_len]
    
    # 전처리 적용
    ko_processed = [preprocess_korean(line) for line in ko_lines]
    en_processed = [preprocess_english(line) for line in en_lines]
    
    # 빈 문장 제거
    valid_pairs = []
    for ko, en in zip(ko_processed, en_processed):
        if ko.strip() and en.strip():  # 둘 다 비어있지 않은 경우만
            valid_pairs.append((ko, en))
    
    # max_samples 적용
    if max_samples and len(valid_pairs) > max_samples:
        valid_pairs = valid_pairs[:max_samples]
    
    # 데이터프레임 생성
    df = pd.DataFrame(valid_pairs, columns=['korean', 'english'])
    
    print(f"총 {len(df)} 개의 문장 쌍이 로드되었습니다.")
    
    return df

In [10]:
def save_corpus_files(df, ko_output_path="ko_corpus.txt", en_output_path="en_corpus.txt"):
    """
    SentencePiece 훈련을 위한 코퍼스 파일 저장
    """
    # 한국어 코퍼스 저장
    df["korean"].to_csv(ko_output_path, index=False, header=False, sep="\n", encoding="utf-8")
    
    # 영어 코퍼스 저장  
    df["english"].to_csv(en_output_path, index=False, header=False, sep="\n", encoding="utf-8")
    
    print(f"코퍼스 파일이 저장되었습니다: {ko_output_path}, {en_output_path}")

In [11]:
# 테스트용 샘플 데이터
test_ko = "개인용 컴퓨터 사용의 상당 부분은 \"이것보다 뛰어날 수 있느냐?\""
test_en = "Much of personal computing is about \"can you top this?\""

print("=== 전처리 테스트 ===")
print(f"원본 한국어: {test_ko}")
print(f"전처리 후: {preprocess_korean(test_ko)}")
print()
print(f"원본 영어: {test_en}")
print(f"전처리 후: {preprocess_english(test_en)}")

# 실제 사용 시:
df = load_and_preprocess_data("korean-english-park.train.ko", 
                              "korean-english-park.train.en", 
                              max_samples=None)
save_corpus_files(df)

=== 전처리 테스트 ===
원본 한국어: 개인용 컴퓨터 사용의 상당 부분은 "이것보다 뛰어날 수 있느냐?"
전처리 후: 개인용 컴퓨터 사용의 상당 부분은 이것보다 뛰어날 수 있느냐 ?

원본 영어: Much of personal computing is about "can you top this?"
전처리 후: much of personal computing is about can you top this ?
총 94100 개의 문장 쌍이 로드되었습니다.
코퍼스 파일이 저장되었습니다: ko_corpus.txt, en_corpus.txt


In [12]:
# vocab_size = 3000
pad_id = 0
bos_id = 1
eos_id = 2
unk_id = 3

spm.SentencePieceTrainer.train(
    input = "ko_corpus.txt",
    model_prefix = "encoder_spm",
    # vocab_size = vocab_size,
    pad_id = pad_id,
    bos_id = bos_id,
    eos_id = eos_id,
    unk_id = unk_id
)

spm.SentencePieceTrainer.train(
    input = "en_corpus.txt",
    model_prefix = "decoder_spm",
    # vocab_size = vocab_size,
    pad_id = pad_id,
    bos_id = bos_id,
    eos_id = eos_id,
    unk_id = unk_id
)

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: ko_corpus.txt
  input_format: 
  model_prefix: encoder_spm
  model_type: UNIGRAM
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 3
  bos_id: 1
  eos_id: 2
  pad_id: 0
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  di

In [13]:
encoder_tokenizer = spm.SentencePieceProcessor()
encoder_tokenizer.load("encoder_spm.model")

decoder_tokenizer = spm.SentencePieceProcessor()
decoder_tokenizer.load("decoder_spm.model")

True

In [14]:
df

,korean,english
0,개인용 컴퓨터 사용의 상당 부분은 이것보다 뛰어날 수 있느냐 ?,much of personal computing is about can you to...
1,모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하...,so a mention a few weeks ago about a rechargea...
2,그러나 이것은 또한 책상도 필요로 하지 않는다 .,"like all optical mice , but it also doesn t ne..."
3,". 달러하는 이 최첨단 무선 광마우스는 허공에서 팔목 , 팔 , 그외에 어떤 부분...",uses gyroscopic sensors to control the cursor ...
4,정보 관리들은 동남 아시아에서의 선박들에 대한 많은 테러 계획들이 실패로 돌아갔...,intelligence officials have revealed a spate o...
...,...,...
94095,우리는 월 일 김승연 회장과 그의 아들이 보복폭행에 가담한 혐의를 찾기 위해 총...,we are hoping to seize material evidence to pr...
94096,월요일 술집 종업원 명은 김회장과 아들에게 폭행을 당했음을 진술했다고 경찰은 말했다 .,"on monday , police secured statements from six..."
94097,그러나 불충분한 증거 확보로 수사에 어려움이 있다 .,but the lack of material evidence is making it...
94098,김회장과 그의 아들은 보복폭행 혐의를 강력히 부인하고 있다 .,kim and his son both deny the allegations .


In [15]:
kor_sample = df["korean"][10000]
eng_sample = df["english"][10000]
print(kor_sample)
print(eng_sample)

그는 민주당 대선후보가 결정 될 때까지 경선을 계속 치르겠다는 뜻을 밝힌 바 있다 .
clinton has vowed to stay in the race until someone gets enough delegates to clinch the nomination .


In [16]:
enc_token = encoder_tokenizer.encode(kor_sample)
enc_token = [encoder_tokenizer.bos_id()] + enc_token + [encoder_tokenizer.eos_id()]
enc_token

[1,
 22,
 8,
 352,
 1380,
 13,
 318,
 190,
 1922,
 1527,
 6,
 283,
 3596,
 2311,
 8,
 1217,
 6,
 1690,
 116,
 26,
 5,
 2]

In [17]:
enc_decoding = encoder_tokenizer.decode(enc_token)
enc_decoding

'그는 민주당 대선후보가 결정 될 때까지 경선을 계속 치르겠다는 뜻을 밝힌 바 있다 .'

In [18]:
class TranslationDataset(Dataset):
    def __init__(self, data, encoder_tokenizer, decoder_tokenizer, max_len):
        self.data = data
        self.encoder_tokenizer = encoder_tokenizer
        self.decoder_tokenizer = decoder_tokenizer
        self.max_len = max_len
        self.pad_id = 0
        self.bos_id = 1
        self.eos_id = 2

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        src_text = self.data.iloc[idx]['korean']
        trg_text = self.data.iloc[idx]['english']

        src_ids = self.encoder_tokenizer.encode(src_text)
        trg_ids = self.decoder_tokenizer.encode(trg_text)

        src_ids = src_ids[:self.max_len]

        # Decoder의 입력에는 START_TOKEN과 END_TOKEN을 추가해줍니다. 단, 최대 길이 제한을 적용시킵니다.
        trg_input = [self.bos_id] + trg_ids[:self.max_len - 2] + [self.eos_id]
        trg_label = trg_ids[:self.max_len - 1] + [self.eos_id]

        # 길이가 짧은 경우 PAD_TOKEN을 추가해줍니다.
        src_ids = src_ids + [self.pad_id] * (self.max_len - len(src_ids))
        trg_input = trg_input + [self.pad_id] * (self.max_len - len(trg_input))
        trg_label = trg_label + [self.pad_id] * (self.max_len - len(trg_label))

        return torch.tensor(src_ids), torch.tensor(trg_input), torch.tensor(trg_label)

In [19]:
train_ratio = 0.8  # 전체 길이의 80%
MAX_LEN = 30 # 임의의 값
BATCH_SIZE = 64

train_data = df.sample(frac=train_ratio, random_state=42)  # 70% 훈련 데이터
valid_data = df.drop(train_data.index)

train_data.reset_index(drop=True, inplace=True)
valid_data.reset_index(drop=True, inplace=True)

train_data = TranslationDataset(train_data, encoder_tokenizer, decoder_tokenizer, max_len=MAX_LEN)
validataion_data = TranslationDataset(valid_data, encoder_tokenizer, decoder_tokenizer, max_len=MAX_LEN)

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(validataion_data, batch_size=BATCH_SIZE, shuffle=False)

In [20]:
for src, trg_input, trg_label in train_loader:
    print(src.shape, trg_input.shape, trg_label.shape)
    break

torch.Size([64, 30]) torch.Size([64, 30]) torch.Size([64, 30])


In [21]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()

        self.W1 = nn.Linear(hidden_dim, hidden_dim)
        self.W2 = nn.Linear(hidden_dim, hidden_dim)
        self.v = nn.Linear(hidden_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        # hidden: (batch_size, hidden_dim)
        # encoder_outputs: (src_len, batch_size, hidden_dim)

        src_len = encoder_outputs.shape[0]

        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)  # (batch_size, src_len, hidden_dim)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)  # (batch_size, src_len, hidden_dim)

        energy = torch.tanh(self.W1(encoder_outputs) + self.W2(hidden))  # (batch_size, src_len, hidden_dim)
        attention = self.v(energy).squeeze(2)  # (batch_size, src_len)

        return nn.functional.softmax(attention, dim=1)  # (batch_size, src_len)

In [22]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, hidden_dim)

    def forward(self, src):
        # src : (src_len, batch_size)
        embedded = self.embedding(src)  # embedded : (src_len, batch_size, emb_dim)
        outputs, hidden = self.rnn(embedded)  # outputs : (src_len, batch_size, hidden_dim)

        return outputs, hidden

In [23]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_dim, attention):
        super(Decoder, self).__init__()

        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)
        # Decoder RNN에는 embedding만 입력
        self.rnn = nn.GRU(emb_dim, hidden_dim)
        # 출력층에는 hidden state와 attention value가 결합되어 입력
        self.fc_out = nn.Linear(hidden_dim + hidden_dim, output_dim)

    def forward(self, input, hidden, encoder_outputs):
        # input : (batch_size,)
        # hidden : (batch_size, hidden_dim)
        # encoder_outputs : (src_len, batch_size, hidden_dim)
        input = input.unsqueeze(0)  # input : (1, batch_size)
        embedded = self.embedding(input)  # embedded : (1, batch_size, emb_dim)

        # attention distribution을 계산합니다. decoder의 이전 hidden state, s_{t-1}와 encoder의 H가 입력됩니다.
        a = self.attention(hidden[-1], encoder_outputs)  # a : (batch_size, src_len)

        # H에 가중치를 부여해 attention value(Context vector) 계산
        a = a.unsqueeze(1)  # a : (batch_size, 1, src_len)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)  # encoder_outputs : (batch_size, src_len, hidden_dim)
        context = torch.bmm(a, encoder_outputs)  # context : (batch_size, 1, hidden_dim)
        context = context.permute(1, 0, 2)  # context : (1, batch_size, hidden_dim)

        output, hidden = self.rnn(embedded, hidden)

        # 출력층에서는 현재 hidden state와 context vector를 결합하여 예측값 생성
        output = output.squeeze(0)  # output : (batch_size, hidden_dim)
        context = context.squeeze(0)  # context : (batch_size, hidden_dim)
        prediction = self.fc_out(torch.cat((output, context), dim=1))  # (batch_size, output_dim)

        return prediction, hidden, a.squeeze(1)

In [24]:
class Seq2SeqAttention(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg=None, max_len=30, bos_id = 1, eos_id=2):
        # 학습 모드에서는 trg_len 사용, 추론 모드에서는 max_len까지 동적 생성
        batch_size = src.shape[1]
        trg_vocab_size = self.decoder.fc_out.out_features

        # 조기 종료를 위해 tensor가 아닌 리스트 사용
        outputs = []

        # 시각화를 위해 attention 저장
        attentions = []

        # 인코더를 통해 context 생성
        encoder_outputs, hidden = self.encoder(src)

        if trg is not None:
            for t in range(0, trg.shape[0]):
                input = trg[t]
                output, hidden, attention = self.decoder(input, hidden, encoder_outputs)
                outputs.append(output.unsqueeze(0))
                attentions.append(attention.unsqueeze(0))

        else:
		    # inference에서는 target(정답)이 없기 때문에 sos_token을 생성해줍니다.
            input = torch.full((batch_size,), bos_id, dtype=torch.long, device=self.device)
            finished = torch.zeros(batch_size, dtype=torch.bool, device=self.device)

            for t in range(max_len):
                output, hidden, attention = self.decoder(input, hidden,  encoder_outputs)
                outputs.append(output.unsqueeze(0))
                attentions.append(attention.unsqueeze(0))
                top1 = output.argmax(1)
                input = top1

                # 조기 종료 조건
                finished |= (top1 == eos_id)
                if finished.all():
                    break

        outputs = torch.cat(outputs, dim=0)  # (trg_len, batch_size, output_dim)
        attentions = torch.cat(attentions, dim=0)  # (trg_len, batch_size, src_len)

        return outputs, attentions

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_dim = len(encoder_tokenizer)
output_dim = len(decoder_tokenizer)
emb_dim = 256
hid_dim = 512

In [26]:
encoder = Encoder(input_dim, emb_dim, hid_dim).to(device)
attention = BahdanauAttention(hid_dim).to(device)
decoder = Decoder(output_dim, emb_dim, hid_dim, attention).to(device)
model = Seq2SeqAttention(encoder, decoder, device).to(device)

In [27]:
print(model)

Seq2SeqAttention(
  (encoder): Encoder(
    (embedding): Embedding(8000, 256)
    (rnn): GRU(256, 512)
  )
  (decoder): Decoder(
    (attention): BahdanauAttention(
      (W1): Linear(in_features=512, out_features=512, bias=True)
      (W2): Linear(in_features=512, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(8000, 256)
    (rnn): GRU(256, 512)
    (fc_out): Linear(in_features=1024, out_features=8000, bias=True)
  )
)


## 4. 훈련하기

### (1) Optimizer & Loss

In [28]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss(ignore_index=pad_id)

print("슝~")

슝~


### (2) train_step 구현하기

In [29]:
def train_step(model, data_loader, optimizer, criterion, epoch):
    model.train()
    epoch_loss = 0

    progress_bar = tqdm(data_loader, desc=f"Epoch {epoch+1}", leave=True)

    for src, trg_input, trg_label in progress_bar:
        src, trg_input, trg_label = src.to(device), trg_input.to(device), trg_label.to(device)
        optimizer.zero_grad()

        outputs,_ = model(src, trg_input)

        # (trg_len, batch_size, output_dim)을 (batch_size * trg_len, output_dim)으로 변환
        outputs = outputs.view(-1, outputs.shape[-1])
        trg_label = trg_label.view(-1)

        loss = criterion(outputs, trg_label)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        optimizer.step()

        epoch_loss += loss.item()

        progress_bar.set_postfix(loss=loss.item())

    return epoch_loss / len(data_loader)

print("슝~")


슝~


### (3) 훈련 시작하기

In [30]:
%%time

EPOCHS = 10

for epoch in range(EPOCHS):
    train_loss = train_step(model, train_loader, optimizer, criterion, epoch)
    print(f'Epoch {epoch+1}/{EPOCHS}, Train Loss: {train_loss:.4f}')

Epoch 2: 100%|██████████| 1177/1177 [04:45<00:00,  4.12it/s, loss=4.82]


Epoch 2/10, Train Loss: 4.7719


Epoch 3: 100%|██████████| 1177/1177 [04:45<00:00,  4.13it/s, loss=5.01]


Epoch 3/10, Train Loss: 4.6379


Epoch 4:  74%|███████▎  | 866/1177 [03:30<01:15,  4.13it/s, loss=4.62]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Epoch 6: 100%|██████████| 1177/1177 [04:45<00:00,  4.12it/s, loss=4.63]


...

Epoch 7/10, Train Loss: 4.4440


Epoch 8:  26%|██▌       | 303/1177 [01:13<03:31,  4.14it/s, loss=4.27]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Epoch 9: 100%|██████████| 1177/1177 [04:45<00:00,  4.12it/s, loss=4.6] 


Epoch 9/10, Train Loss: 4.3974


Epoch 10: 100%|██████████| 1177/1177 [04:45<00:00,  4.12it/s, loss=4.54]

Epoch 10/10, Train Loss: 4.3794
CPU times: user 44min 16s, sys: 3min 25s, total: 47min 41s
Wall time: 47min 31s


In [31]:
def eval_step(model, data_loader, optimizer, criterion):
    model.eval()
    total_loss = 0

    for src, trg_input, trg_label in data_loader:
        src, trg_input, trg_label = src.to(device), trg_input.to(device), trg_label.to(device)

        outputs, _ = model(src, trg_input)

        # (trg_len, batch_size, output_dim)을 (batch_size * trg_len, output_dim)으로 변환
        outputs = outputs.view(-1, outputs.shape[-1])
        trg_label = trg_label.view(-1)

        loss = criterion(outputs, trg_label)

        total_loss += loss.item()

    return total_loss / len(data_loader)

print("슝~")

슝~


In [ ]:
%%time

EPOCHS = 20

for epoch in range(EPOCHS):
    train_loss = train_step(model, train_loader, optimizer, criterion, epoch)
    valid_loss = eval_step(model, validation_loader, optimizer, criterion)
    print(f'Epoch {epoch+1}/{EPOCHS}, Train Loss: {train_loss:.4f}, Validation Loss: {valid_loss:.4f}')

Epoch 1: 100%|██████████| 1177/1177 [04:45<00:00,  4.12it/s, loss=4.33]


Epoch 1/20, Train Loss: 4.3623, Validation Loss: 4.7544


Epoch 2:  27%|██▋       | 321/1177 [01:18<03:27,  4.12it/s, loss=4.26]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Epoch 3: 100%|██████████| 1177/1177 [04:46<00:00,  4.11it/s, loss=4.24]


Epoch 3/20, Train Loss: 4.3331, Validation Loss: 4.7671


Epoch 4: 100%|██████████| 1177/1177 [04:46<00:00,  4.11it/s, loss=4.64]


Epoch 4/20, Train Loss: 4.3208, Validation Loss: 4.7674


Epoch 5:  27%|██▋       | 319/1177 [01:17<03:29,  4.10it/s, loss=4.35]

In [ ]:
def evaluate(sentence, model, encoder_tokenizer, decoder_tokenizer, max_len=30):
    model.eval()

    sentence = preprocess_korean(sentence)
    src_ids = encoder_tokenizer.encode(sentence)
    src_ids = src_ids[:max_len]
    src_ids = src_ids + [0] * (max_len - len(src_ids))  # 패딩 추가
    src_tensor = torch.tensor(src_ids).unsqueeze(1).to(device)  # (src_len, 1)

    with torch.no_grad():
        outputs, attentions = model(src_tensor, max_len=max_len)

    result = [decoder_tokenizer.decode([token.item()]) for token in outputs.argmax(2).squeeze(1)]

    if "<end>" in result:
        result = result[:result.index("<end>")]

    return result, sentence, attentions.squeeze(1).cpu().numpy()

In [ ]:
def plot_attention(attention, sentence, predicted_sentence):
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticks(range(len(sentence)))
    ax.set_xticklabels(sentence, fontdict=fontdict, rotation=90)

    ax.set_yticks(range(len(predicted_sentence)))
    ax.set_yticklabels(predicted_sentence, fontdict=fontdict)

    plt.show()

In [ ]:
def translate(sentence, model, encoder_tokenizer, decoder_tokenizer, max_len=30):
    result, sentence, attention = evaluate(sentence, model, encoder_tokenizer, decoder_tokenizer, max_len)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

    # Attention 크기 조정 (trg_len, src_len)
    attention = attention[:len(result), :len(sentence.split())]

    plot_attention(attention, sentence.split(), result)

In [ ]:
translate("가장 쌘놈이 모든걸 가진다.", model, encoder_tokenizer, decoder_tokenizer, max_len=30)

In [ ]:
translate("도와 드릴까요?", model, encoder_tokenizer, decoder_tokenizer, max_len=30)

In [ ]:
# 실행 예제
translate("커피 한잔 마실 수 있을까요?", model, encoder_tokenizer, decoder_tokenizer, max_len=30)